In [1]:
from typing import List
import numpy as np

from pyquil import Program
from pyquil.gates import MEASURE, I, CNOT, X, H, CZ, RY
from pyquil.quil import address_qubits
from pyquil.quilatom import QubitPlaceholder
from pyquil.api import QVMConnection
from pyquil.noise import pauli_kraus_map, dephasing_kraus_map

In [2]:
qvm = QVMConnection()

In [3]:
def NN_scheme_encode(qubit: QubitPlaceholder, N: int) -> (Program, List[QubitPlaceholder]):
    
    ### qubit: qubit you want to encode (main qubit)
    ### N: number of qubits you want to encode the main qubit in. 
    ### For N=1, there is no encoding
    
    code_register = QubitPlaceholder.register(N)  # the List[QubitPlaceholder] of the qubits you have encoded into
    code_register[0] = qubit
    
    pq = Program()
    
    ### creation of GHZ state: 
    for ii in range(N-1):
        pq += CNOT(code_register[ii],code_register[ii+1])
        
        
    for jj in range(N-1):
        pq += H(code_register[jj])
        pq += CZ(code_register[jj+1],code_register[jj])
        
    
    for kk in range(N-1):
        pq += CNOT(code_register[kk],code_register[-1])

    return pq, code_register


def NN_scheme_decode(code_register: List[QubitPlaceholder]) -> (Program, QubitPlaceholder):
    
    out = QubitPlaceholder()
    n = len(code_register)
    
    pq = Program()
    
    ### This is where the magic happens
    for ii in range(n):
        pq += CNOT(code_register[ii],out)
        
        
    return pq, out


def NN_scheme_test(qubit, prep_program, N_encode, N_decode):
    
    ### qubit: qubit we want to encode (main qubit)
    ### prep_program: arbitrary program to put the main qubit in the state we want to 
    ### transmit
    ### N_encode: number of qubits to encode the main qubit in
    ### N_decode: number of qubits to read out
    ### if N_decode < N_encode, we will always get alpha squared = beta squared = 0.5,
    ### no matter what we originally encoded the qubit in.
    ### Note that this test only gives the absolute squared values of alpha and beta

    pq = Program()
    pq += prep_program
        
    prog, code_register = NN_scheme_encode(qubit,N_encode)

    pq += prog

    progg, out = NN_scheme_decode(code_register[0:N_decode])

    pq += progg

    ro = pq.declare('ro', 'BIT', 1)
    pq += MEASURE(out, ro)

    result = np.sum(qvm.run(address_qubits(pq), trials=1000))/1000.0
    
    alpha_sqrd = 1.0 - result
    beta_sqrd = result
    
    print('alpha squared = {}'.format(alpha_sqrd))
    print('beta squared = {}'.format(beta_sqrd))
    
    return alpha_sqrd, beta_sqrd

In [6]:
test = QubitPlaceholder()
pq = Program()
# pq += X(XX)

pq += RY(2.0*np.pi/3.0,test)

N_encode, N_decode = 4, 4

alpha_sqrd, beta_sqrd = NN_scheme_test(test, pq, N_encode, N_decode)

alpha squared = 0.236
beta squared = 0.764
